In [2]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools

In [19]:
class Person():

    def __init__(self, num_of_issues, opinions=None, weights=None):

        self.p = pd.DataFrame()
        self.total_issues = num_of_issues
        self.p.index = range(num_of_issues)
        self.p.index.name = 'Issues'

        rng = np.random.default_rng()

        if opinions is None:
            self.p['Opinion'] = [rng.normal(0,1) for _ in range(num_of_issues)]
        else: 
            self.p['Opinion'] = opinions

        if weights is None:
            self.p['Weight'] = [rng.normal(0,1)**2 for _ in range(num_of_issues)]
        else:
            self.p['Weight'] = weights


        self.p['Rhetoric'] = np.zeros(self.total_issues)


    def becomeVocal(self, vocality=0.25, num_of_issues=None):

        if num_of_issues is None:
            num_of_issues = int(self.total_issues*vocality)
        else:
            num_of_issues = min(num_of_issues, self.total_issues)

        sorted_issues = np.argsort(self.p['Weight'].to_numpy())[::-1][:num_of_issues]

        for issue in sorted_issues:
            issue = int(issue)
            self.p.loc[issue, 'Rhetoric'] = self.p.loc[issue, 'Weight']

    def returnFrame(self):
        return self.p
    
    def findClosestCanidate(self, canidates):

        distances_per_canidate = []
        for canidate in canidates:
            distance_per_issue = []
            for issue in self.p.index:
                opinion_distance = np.abs(self.p.loc[issue, 'Opinion'] - canidate.p.loc[issue, 'Opinion'])
                conviction_distance = np.abs(self.p.loc[issue, 'Weight'] - canidate.p.loc[issue, 'Weight'])
                opinion_power = 1 + (1 / (1 + np.abs(self.p.loc[issue, 'Opinion'] - canidate.p.loc[issue, 'Opinion'])))**10
                conviction_power = 1 + np.sqrt(self.p.loc[issue, 'Weight'] * canidate.p.loc[issue, 'Weight'])

                distance = (opinion_distance**conviction_power)*(opinion_power**conviction_distance)
                distance_per_issue.append(distance)

            distances_per_canidate.append(sum([x**2 for x in distance_per_issue]))

        return distances_per_canidate

    def __str__(self) -> str:
        return str(self.p)
    
    
    def influenceOthers(self, people, influence_matrix, influence_function):
        '''
        influence_matrix is a n x m matrix
        where n is number of people
        and m is number of issues
        
        '''
        for i, person in enumerate(people):
            influence_function(self, person, influence_matrix[i])
        pass


In [20]:
for i, (voter_op, voter_weight, canidate_op, canidate_weight) in enumerate(itertools.product([-1], [5, 0.1], [0.9, -0.9], [5, 0.1])):

    print(f"{i+1}\tAgree: {'Yes' if voter_op-canidate_op > -1 else 'No'}\tVoter Conviction: {voter_weight}\tCanidate Conviction: {canidate_weight}")

    person = Person(1, opinions=[voter_op], weights=[voter_weight])
    #print(person)
    canidate = Person(1, opinions=[canidate_op], weights=[canidate_weight])
    #print(person)
    print(person.findClosestCanidate([canidate]))

    print('\n')
    



1	Agree: No	Voter Conviction: 5	Canidate Conviction: 5
[np.float64(2213.3149190661597)]


2	Agree: No	Voter Conviction: 5	Canidate Conviction: 0.1
[np.float64(8.950050561046165)]


3	Agree: Yes	Voter Conviction: 5	Canidate Conviction: 5
[np.float64(9.999999999999974e-13)]


4	Agree: Yes	Voter Conviction: 5	Canidate Conviction: 0.1
[np.float64(0.00941159890483724)]


5	Agree: No	Voter Conviction: 0.1	Canidate Conviction: 5
[np.float64(8.950050561046165)]


6	Agree: No	Voter Conviction: 0.1	Canidate Conviction: 0.1
[np.float64(4.1044779046045985)]


7	Agree: Yes	Voter Conviction: 0.1	Canidate Conviction: 5
[np.float64(0.00941159890483724)]


8	Agree: Yes	Voter Conviction: 0.1	Canidate Conviction: 0.1
[np.float64(0.006309573444801926)]




In [104]:
0.1**1.5

0.0316227766016838

In [ ]:
issues = []*10
for i in range(len(issues)):